In [6]:
import os
import cv2
import pytesseract

# Dossier contenant les sous-dossiers de régions recadrées
input_folder = 'cropped_regions'
output_folder = 'ocr_results'
os.makedirs(output_folder, exist_ok=True)  # Créer un dossier pour les résultats OCR

# Fonction de prétraitement (améliore souvent les résultats OCR)
def preprocess_image(image):
     # Redimensionner pour agrandir les petits textes
    scale = 5  # Augmenter la taille de l'image
    height, width, _ = image.shape
    resized_image = cv2.resize(image, (width * scale, height * scale), interpolation=cv2.INTER_CUBIC)
    
    # Appliquer une binarisation avec seuil adaptatif sur chaque canal de couleur
    binary = cv2.adaptiveThreshold(
        cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 31, 10
    )
    
    return binary

# Fonction pour appliquer l'OCR sur une image
def extract_text_from_image(image_path):
    # Charger l'image
    image = cv2.imread(image_path)

    # Appliquer le prétraitement
    processed_image = preprocess_image(image)

    # Configurer les paramètres de Tesseract
    custom_config = r'--oem 3 --psm 6'  # OEM 3 : Mode LSTM, PSM 6 : Une seule colonne
    text = pytesseract.image_to_string(processed_image, lang='eng', config=custom_config)
    
    return text.strip()

# Parcourir chaque sous-dossier dans le dossier des régions recadrées
for folder_name in os.listdir(input_folder):
    folder_path = os.path.join(input_folder, folder_name)
    
    if os.path.isdir(folder_path):  # Vérifier si c'est un dossier
        print(f"Traitement des régions dans le dossier : {folder_name}")
        
        # Créer un sous-dossier pour les résultats OCR de ce dossier
        folder_output_path = os.path.join(output_folder, folder_name)
        os.makedirs(folder_output_path, exist_ok=True)
        
        # Parcourir chaque fichier JPEG dans le sous-dossier
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
                input_path = os.path.join(folder_path, file_name)
                
                # Appliquer l'OCR
                print(f"Traitement de l'image : {file_name}")
                text = extract_text_from_image(input_path)

                # Afficher le texte extrait
                print(f"Texte extrait :\n{text}")
                print("-" * 50)

                # Sauvegarder le texte dans un fichier
                output_text_path = os.path.join(folder_output_path, f"{os.path.splitext(file_name)[0]}.txt")
                with open(output_text_path, 'w', encoding='utf-8') as f:
                    f.write(text)

                # Facultatif : Sauvegarder l'image prétraitée
                processed_image = preprocess_image(cv2.imread(input_path))
                output_image_path = os.path.join(folder_output_path, f"{os.path.splitext(file_name)[0]}_processed.jpg")
                cv2.imwrite(output_image_path, processed_image)

print("OCR terminé. Les textes extraits et les images prétraitées sont sauvegardés dans :", output_folder)


Traitement des régions dans le dossier : DS_17
Traitement de l'image : region_2.jpg
Texte extrait :
\ 2. CAN'T
PASS ANY
INFORMA}-
TION NOW
--------------------------------------------------
Traitement de l'image : region_0.jpg
Texte extrait :
i |
aft
TENGEN-
SAMA TS...
y
--------------------------------------------------
Traitement de l'image : region_1.jpg
Texte extrait :
' u

NOR CANT |\
CONTACT THE
OTHER TV/O.
--------------------------------------------------
Traitement de l'image : region_3.jpg
Texte extrait :
/
c TO
GET OUT...
--------------------------------------------------
Traitement des régions dans le dossier : DS_42
Traitement de l'image : region_2.jpg
Texte extrait :
f FROM NOW ON,
YOU GUYS. SHOULD
SERVE ME WITH
MORE SUICIDAL

«DEVOTION.

-\ |
--------------------------------------------------
Traitement de l'image : region_0.jpg
Texte extrait :
{DONT ou N
{ EXCITED AND GIVE
| = ME INFORMATION
| «IF YOUHAVENT =f
) EVEN CONFIRMED — /:
\ IT YET. Lo
-------------------------

In [7]:
import os
import re

# Dossier contenant les résultats OCR
ocr_results_folder = "ocr_results"

# Fonction pour nettoyer le texte : garder uniquement les lettres majuscules
def clean_text(text):
    # Utiliser une expression régulière pour supprimer tout sauf les lettres majuscules
    cleaned_text = re.sub(r"[^A-Z\s]", "", text)  # Garder A-Z et les espaces
    return cleaned_text.strip()

# Parcourir tous les sous-dossiers et fichiers dans ocr_results
for folder_name in os.listdir(ocr_results_folder):
    folder_path = os.path.join(ocr_results_folder, folder_name)

    if os.path.isdir(folder_path):  # Vérifier si c'est un sous-dossier
        print(f"Traitement du dossier : {folder_name}")
        
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".txt"):  # Vérifier si c'est un fichier texte
                file_path = os.path.join(folder_path, file_name)
                print(f"Nettoyage du fichier : {file_name}")
                
                # Lire le contenu du fichier
                with open(file_path, "r", encoding="utf-8") as f:
                    original_text = f.read()

                # Nettoyer le texte
                cleaned_text = clean_text(original_text)

                # Sauvegarder le texte nettoyé dans le même fichier
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(cleaned_text)

print("Nettoyage terminé. Les fichiers ont été mis à jour.")


Traitement du dossier : DS_17
Nettoyage du fichier : region_2.txt
Nettoyage du fichier : region_1.txt
Nettoyage du fichier : region_0.txt
Nettoyage du fichier : region_3.txt
Traitement du dossier : DS_42
Nettoyage du fichier : region_2.txt
Nettoyage du fichier : region_1.txt
Nettoyage du fichier : region_0.txt
Nettoyage du fichier : region_3.txt
Traitement du dossier : DS_69
Nettoyage du fichier : region_6.txt
Nettoyage du fichier : region_4.txt
Nettoyage du fichier : region_8.txt
Nettoyage du fichier : region_2.txt
Nettoyage du fichier : region_7.txt
Nettoyage du fichier : region_1.txt
Nettoyage du fichier : region_5.txt
Nettoyage du fichier : region_0.txt
Nettoyage du fichier : region_9.txt
Nettoyage du fichier : region_3.txt
Traitement du dossier : DS_50
Nettoyage du fichier : region_4.txt
Nettoyage du fichier : region_2.txt
Nettoyage du fichier : region_1.txt
Nettoyage du fichier : region_0.txt
Nettoyage du fichier : region_3.txt
Traitement du dossier : DS_93
Nettoyage du fichier :

In [8]:
from transformers import MarianMTModel, MarianTokenizer
import os

# Charger le modèle de traduction anglais -> français
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Fonction pour traduire un texte
def translate_to_french(text):
    # Préparer le texte pour la traduction
    tokenized_text = tokenizer.prepare_seq2seq_batch([text], return_tensors="pt", padding=True)
    translated = model.generate(**tokenized_text)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Dossier contenant les fichiers OCR
ocr_input_folder = "ocr_results"
translated_output_folder = "translated_texts"
os.makedirs(translated_output_folder, exist_ok=True)

# Traduire chaque fichier texte extrait
for folder_name in os.listdir(ocr_input_folder):
    folder_path = os.path.join(ocr_input_folder, folder_name)
    
    if os.path.isdir(folder_path):  # Vérifier si c'est un dossier
        print(f"Traitement des fichiers OCR dans le dossier : {folder_name}")
        
        # Créer un sous-dossier pour les textes traduits
        output_folder_path = os.path.join(translated_output_folder, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)
        
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.txt'):
                input_file_path = os.path.join(folder_path, file_name)
                
                # Lire le texte extrait
                with open(input_file_path, 'r', encoding='utf-8') as file:
                    english_text = file.read()
                
                print(f"Texte en anglais :\n{english_text}")
                
                # Traduire en français
                translated_text = translate_to_french(english_text)
                print(f"Texte traduit :\n{translated_text}")
                
                # Sauvegarder le texte traduit
                output_file_path = os.path.join(output_folder_path, file_name)
                with open(output_file_path, 'w', encoding='utf-8') as file:
                    file.write(translated_text)

print("Traduction terminée. Les textes traduits sont sauvegardés dans :", translated_output_folder)


Note: you may need to restart the kernel to use updated packages.


/home/shadowsun/master/manga-yolo/myenv/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Traitement des fichiers OCR dans le dossier : DS_17
Texte en anglais :
CANT
PASS ANY
INFORMA
TION NOW


/home/shadowsun/master/manga-yolo/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4079: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Texte traduit :
CANT PASSE TOUTES LES INFORMATIONS MAINTENANT
Texte en anglais :
NOR CANT 
CONTACT THE
OTHER TVO
Texte traduit :
NOR CANT CONTACTER L'Autre TVO
Texte en anglais :
TENGEN
SAMA TS
Texte traduit :
TENGEN SAMA TS
Texte en anglais :
TO
GET OUT
Texte traduit :
Pour sortir
Traitement des fichiers OCR dans le dossier : DS_42
Texte en anglais :
FROM NOW ON
YOU GUYS SHOULD
SERVE ME WITH
MORE SUICIDAL

DEVOTION
Texte traduit :
À partir de maintenant, vous devriez me servir avec plus de défiance
Texte en anglais :
TAMATTHEHEIGHT 
 OF MY DISPLEASURE

 BECAUSE ITS BEEN
  ONEHUNDRED AND
 THIRTEEN YEARS
 SINCE AN UPPER

 MOON WAS KILLED
Texte traduit :
Tamattheheight de mon déplaisir parce qu'il a été tué pendant des années et trois années après qu'un loup a été tué.
Texte en anglais :
DONT  N
 EXCITED AND GIVE
  ME INFORMATION
 IF YOUHAVENT 
 EVEN CONFIRMED  
 IT YET L
Texte traduit :
DONT N'EXCITES ET MONTENT DES INFORMATIONS SI VOUS AVEZ ÉGALEMENT CONFIRMÉ
Texte en anglais :
SPOILED